In [264]:
## Global Options
rate_limit_seconds = 3  # API Rate limit in seconds
rate_limit_timeout = 0  # When the rate limit will expire

def divider(): 
    print("-" * 50)


In [265]:
import os
import time
from openai import OpenAI

if not os.environ.get("OPENAI_API"):
    raise ValueError("OPENAI_API environment variable is not set. Please set it to your API key.")
else : 
    print("API Key is set.")

# -------------------------------------------------
# Function to generate a response 

## Models to test with gpt-5, gpt-5-chat-latest (chatty), gpt-5-nano (cheap reasoning, no temperature)
# seed=seed  # Not supported with client.responses.create

def generate_response(prompt, model="gpt-5", temperature=None, max_tokens=1000, top_p=None, debug=False):

    if debug:
        print(f"Generating response with model: {model}, temperature: {temperature}, max_tokens: {max_tokens}, top_p: {top_p}, ")

    # Adjust parameters for models
    if model == "gpt-5-nano":
        if max_tokens < 1000 : 
            max_tokens = 1000    # requires at least 1000 tokens
            if debug: print("Max Tokens set to 1000")
        if temperature != None: 
            temperature = None  # not supported
            if debug: print("Temperature Not Supported")
        if top_p != None: 
            top_p = None              # not supported
            if debug: print("Top_p Not Supported")
    if model == "gpt-5" :
        if max_tokens < 1000 : 
            max_tokens = 1000    # requires at least 1000 tokens
            if debug: print("Max Tokens set to 1000")
        if temperature != None:
            temperature = None  # not supported
            if debug: print("Temperature Not Supported")
        if top_p != None: 
            top_p = None              # not supported
            if debug: print("Top_p Not Supported")


 
    # Rate limit
    global rate_limit_seconds
    global rate_limit_timeout
    now = time.time()
    dif = rate_limit_timeout - now   

    # Spinner 
    if (dif > 0):
        # Spinner Countdown   
        while dif > 0:
            print(f"\rRate limit {dif:5.3f}s", end='', flush=True)
            time.sleep(0.25)  # Update spinner every 0.25 seconds
            dif = rate_limit_timeout - time.time()
        print("\r                         \r", end='', flush=True)
        # /Spinner 

    # Set the Rate Limit    
    rate_limit_timeout = time.time() + rate_limit_seconds
    
    client = OpenAI(api_key=os.environ["OPENAI_API"],
                    )

    try:
        response = client.responses.create(
        model=model,
        input=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_output_tokens=max_tokens,
        top_p=top_p,  
        service_tier="flex",  
        safety_identifier="test_user",
        prompt_cache_key="test_user",
        )
    except Exception as e:
        print(f"Error generating response: {e}")
        return None
    if debug:
        print(f"Usage: {response.usage.input_tokens} + {response.usage.output_tokens} = {response.usage.total_tokens}\n")
    return response.output_text 

# /------------------------------------------------

# Test
if (True):  # Set to True to run examples
    print()
    prompt = "Testing 1, 2, 3, ..."
    response = generate_response(prompt, debug=True)
    print(f"Prompt: {prompt}\nResponse: {response}")
    

API Key is set.

Generating response with model: gpt-5, temperature: None, max_tokens: 1000, top_p: None, 
Usage: 17 + 199 = 216

Prompt: Testing 1, 2, 3, ...
Response: Loud and clear! How can I help today? 
If you’re testing, you could:
- Ask a quick question or summary
- Brainstorm ideas or draft some text
- Request a code snippet or explanation
- Upload an image for analysis
- Do a quick calculation

Want a quick demo of anything?


In [266]:
if (True):  # Set to True to run examples
    # Example: Basic prompt
    basic_prompt = "Explain AI in one sentence."
    response = generate_response(basic_prompt, debug=True)
    print("Basic Prompt")
    print(f"Prompt: {basic_prompt}\nResponse: {response}\n")
    divider()

    # Example: Zero-shot, few-shot, and CoT prompts
    zero_shot_prompt = "Classify this sentiment: 'I love this product!'"
    zero_response = generate_response(zero_shot_prompt, temperature=0.5, debug=True)
    print("Zero-shot Prompt")
    print(f"Prompt: {zero_shot_prompt}\nResponse: {zero_response}\n")
    divider()

    # Few-shot example
    few_shot_prompt = """
    Example 1: Text: 'Great movie.' Sentiment: Positive.
    Example 2: Text: 'Terrible service.' Sentiment: Negative.
    Classify: 'Okay experience.'
    """
    few_response = generate_response(few_shot_prompt, top_p=0.9, debug=True)  # Now supported
    print("Few-shot Prompt")
    print(f"Prompt: {few_shot_prompt}\nResponse: {few_response}\n")
    divider()

    # Chain of Thought (CoT) example
    cot_prompt = "Solve: What is 15% of 200? Think step by step."
    cot_response = generate_response(cot_prompt, temperature=0.2, debug=True)
    print("CoT Prompt")
    print(f"Prompt: {cot_prompt}\nResponse: {cot_response}\n")
    divider()


Generating response with model: gpt-5, temperature: None, max_tokens: 1000, top_p: None, 
Usage: 12 + 222 = 234

Basic Prompt
Prompt: Explain AI in one sentence.
Response: AI is technology that enables computers to learn from data and make decisions or predictions in ways that mimic aspects of human intelligence.

--------------------------------------------------
Generating response with model: gpt-5, temperature: 0.5, max_tokens: 1000, top_p: None, 
Temperature Not Supported
Usage: 17 + 71 = 88      

Zero-shot Prompt
Prompt: Classify this sentiment: 'I love this product!'
Response: Positive

--------------------------------------------------
Generating response with model: gpt-5, temperature: None, max_tokens: 1000, top_p: 0.9, 
Top_p Not Supported
Usage: 49 + 139 = 188    

Few-shot Prompt
Prompt: 
    Example 1: Text: 'Great movie.' Sentiment: Positive.
    Example 2: Text: 'Terrible service.' Sentiment: Negative.
    Classify: 'Okay experience.'
    
Response: Sentiment: Neutral.

In [267]:
print("End of recent edits")

End of recent edits
